# 1. Download Car Model URLs

## 1.1 Saving car model URLs from page 1

In [1]:
import requests
from bs4 import BeautifulSoup

# URL of eBay Motors search results for sedans
#url = 'https://www.ebay.com/sch/Cars-Trucks/6001/i.html?_nkw=sedan&_sop=12'
#url = 'https://www.ebay.com/sch/i.html?_from=R40&_nkw=sedan&_sacat=0&_ipg=240&_dmd=1&rt=nc'
url = 'https://www.ebay.com/sch/6001/i.html?_from=R40&_nkw=sedan&_dmd=1&_ipg=240'

# Send a GET request to the page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # First, find the <ul> element with class "srp-results srp-list clearfix"
    ul_element = soup.find('ul', class_='srp-results srp-list clearfix')

    # Check if we found the correct <ul>
    if ul_element:
        # Find all <li> tags within this <ul> that have the required attributes
        cars = ul_element.find_all('li', attrs={
            'data-viewport': True,
            'id': True,
            'class': True,
            'data-view': True
        })

        # Loop through each <li> tag to extract data
        for car in cars:
            # Extract the car title from the 's-item__title' class
            title_tag = car.find('img')

            # Extract the image URL from the 'img' tag within the 's-item__image-wrapper' class
            image_tag = car.find('img')

            # Check if title and image URL are found, then print them
            if title_tag and image_tag:
                car_title = title_tag['alt']
                car_image = image_tag['src']
                print(f'Car Model: {car_title}, Image URL: {car_image}')
            else:
                print('No title or image found for this entry')
    else:
        print("Could not find the specified <ul> element.")
else:
    print(f"Failed to retrieve page: {response.status_code}")


Car Model: Buick: Grand National GRAND NATIONAL, Image URL: https://i.ebayimg.com/images/g/IY8AAOSwQctnPZD2/s-l500.jpg
Car Model: 2015 Toyota Camry , Image URL: https://i.ebayimg.com/images/g/sZ8AAOSwej5nTiqL/s-l500.jpg
Car Model: 1997 BMW 5-Series 528i, Image URL: https://i.ebayimg.com/images/g/FWYAAOSw5gJnUfli/s-l140.jpg
Car Model: Audi: A4 S-Line, Image URL: https://i.ebayimg.com/images/g/R0EAAOSwEfFnEtR7/s-l140.jpg
Car Model: 2007 Mercedes-Benz S-Class S65, Image URL: https://i.ebayimg.com/images/g/bp8AAOSwsS9nSL-G/s-l140.jpg
Car Model: Mercedes-Benz: E-Class, Image URL: https://i.ebayimg.com/images/g/snIAAOSwIYdnT3zP/s-l140.jpg
Car Model: 1996 Chevrolet Impala , Image URL: https://i.ebayimg.com/images/g/SUMAAOSwWqJnUIob/s-l140.jpg
Car Model: BMW: 5-Series, Image URL: https://i.ebayimg.com/images/g/bmYAAOSwHRtmtBmN/s-l140.jpg
Car Model: 1987 Honda Civic 1.5, Image URL: https://i.ebayimg.com/images/g/jHgAAOSwfiRnS6XH/s-l140.jpg
Car Model: 1986 Mercedes-Benz 190-Series , Image URL: h

In [2]:
import requests
from PIL import Image
from io import BytesIO

# URL of the image
url = "https://i.ebayimg.com/images/g/TrYAAOSw8Y1nGosa/s-l140.jpg"

# Fetch the image
response = requests.get(url)
img = Image.open(BytesIO(response.content))

# Save the image to the current directory
img.save('car.jpg')

# Display the image
img.show()

### Saving the results to csv

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of eBay Motors search results for sedans
url = 'https://www.ebay.com/sch/6001/i.html?_from=R40&_nkw=sedan&_dmd=1&_ipg=240'

# Send a GET request to the page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # First, find the <ul> element with class "srp-results srp-list clearfix"
    ul_element = soup.find('ul', class_='srp-results srp-list clearfix')

    # Check if we found the correct <ul>
    if ul_element:
        # Create an empty list to store the car data
        car_data = []

        # Find all <li> tags within this <ul> that have the required attributes
        cars = ul_element.find_all('li', attrs={
            'data-viewport': True,
            'id': True,
            'class': True,
            'data-view': True
        })

        # Loop through each <li> tag to extract data
        for car in cars:
            # Extract the car title from the 's-item__title' class
            title_tag = car.find('img')

            # Extract the image URL from the 'img' tag within the 's-item__image-wrapper' class
            image_tag = car.find('img')
            
            # Extract URL of the car model
            link_tag = car.find('a', href=True)

            # Check if title and image URL are found, then save them to the list
            if title_tag and image_tag and link_tag:
                car_title = title_tag['alt']
                car_image = image_tag['src']
                car_url = link_tag['href']
                # Append the data as a dictionary
                car_data.append({'Car Model': car_title, 'Image URL': car_image, 'Car URL': car_url})
            else:
                print('No title, image, or URL found for this entry')

        # Convert the list of dictionaries to a pandas DataFrame
        df = pd.DataFrame(car_data)

        # Save the DataFrame to a CSV file
        df.to_csv('ebay_sedans_info.csv', index=False)

        print("Data saved to 'ebay_sedans_pandas.csv' successfully.")
    else:
        print("Could not find the specified <ul> element.")
else:
    print(f"Failed to retrieve page: {response.status_code}")


Data saved to 'ebay_sedans_pandas.csv' successfully.


## 1.2 Saving car model URLs from all pages

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to scrape a single page
def scrape_page(page_number):
    # Modify the URL to include the page number (_pgn parameter for pagination)
    #url = f'https://www.ebay.com/sch/i.html?_from=R40&_nkw=sedan&_sacat=0&_ipg=240&_dmd=1&rt=nc&_pgn={page_number}'
    url = f'https://www.ebay.com/sch/6001/i.html?_from=R40&_nkw=sedan&_dmd=1&_ipg=240&_pgn={page_number}'
    
    # Send a GET request to the page
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # First, find the <ul> element with class "srp-results srp-list clearfix"
        ul_element = soup.find('ul', class_='srp-results srp-list clearfix')

        # Check if we found the correct <ul>
        if ul_element:
            # Create an empty list to store the car data
            car_data = []

            # Find all <li> tags within this <ul> that have the required attributes
            cars = ul_element.find_all('li', attrs={
                'data-viewport': True,
                'id': True,
                'class': True,
                'data-view': True
            })

            # Loop through each <li> tag to extract data
            for car in cars:
                # Extract the car title from the 's-item__title' class
                title_tag = car.find('img')

                # Extract the image URL from the 'img' tag within the 's-item__image-wrapper' class
                image_tag = car.find('img')
                
                # Extract URL of the car model
                link_tag = car.find('a', href=True)

                # Check if title and image URL are found, then save them to the list
                if title_tag and image_tag and link_tag:
                    car_title = title_tag['alt']
                    car_image = image_tag['src']
                    car_url = link_tag['href']
                    # Append the data as a dictionary
                    car_data.append({'Car Model': car_title, 'Image URL': car_image, 'Car URL': car_url})
            
            return car_data

        else:
            print(f"Could not find the specified <ul> element on page {page_number}.")
            return []
    else:
        print(f"Failed to retrieve page {page_number}: {response.status_code}")
        return []

# Loop through pages until there are no more results
page_number = 1
max_page_number = 3 # Set an upperbound for the number of scraping pages to save time if needed
all_car_data = []

while True and page_number <= max_page_number:
    print(f"Scraping page {page_number}...")
    car_data = scrape_page(page_number)

    if not car_data:
        print(f"No more results found on page {page_number}. Stopping.")
        break

    # Append data from the current page to the main list
    all_car_data.extend(car_data)

    page_number += 1

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(all_car_data)

# Save the DataFrame to a CSV file
df.to_csv('ebay_sedans_info.csv', index=False)

print("Data saved to 'ebay_sedans_info.csv' successfully.")

Scraping page 1...
Scraping page 2...
Scraping page 3...
Data saved to 'ebay_sedans_info.csv' successfully.


# 2. Save the URLs of the images

## 2.1 Saving the URLs of the images for 1 car model

In [17]:
import requests
from bs4 import BeautifulSoup

# URL of the eBay listing page
url = 'https://www.ebay.com/itm/135265261254?_skw=sedan&hash=item1f7e7016c6:g:Zo8AAOSwcCFm8c5b&itmprp=enc%3AAQAJAAAA4HoV3kP08IDx%2BKZ9MfhVJKkhtNc4zMnXPU9b4S7dbB%2BvOoqb9cE9OB2%2FllVZWzbjzjp4RL1RVC%2B9fAh2czCDxUl9dQDf5j2xA%2FQLWIhpBtu0%2BAKC7V9Tx5CbM4%2FWWA9IIZfboiMNolv8EcKoP0Hnh03m4BGuSszdj6Vue2cPBtpfcYvSNPanVq0W%2FVQDXQYKyNmQ%2BrUIWRePYCjMRHwoAhT6oKQhxwawqffD9714ot0zCtcXQglVBLxk7CmHHF%2Bjmd51FJYH3Ko4Est--dLCm9Xpw3D%2F6704joE5CuZLb7od%7Ctkp%3ABk9SR-qzjL_KZA'

# Send a GET request to the page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all <div> elements with the specified class that contains the image carousel items
    image_divs = soup.find_all('div', class_="ux-image-carousel-item image-treatment image")

    # Use a set to store unique image URLs
    image_urls = set()

    # Extract the URLs from the <img> tags within these <div> elements
    for div in image_divs:
        img_tag = div.find('img')  # Find the img tag within the div

        # Extract src, data-zoom-src, and srcset if available
        if img_tag:
            if 'src' in img_tag.attrs:
                image_urls.add(img_tag['src'])  # Add src to the set (avoiding duplicates)
            if 'data-zoom-src' in img_tag.attrs:
                image_urls.add(img_tag['data-zoom-src'])  # Add data-zoom-src to the set
            if 'srcset' in img_tag.attrs:
                # srcset can contain multiple resolutions; split them and add each one
                srcset_urls = [url.split()[0] for url in img_tag['srcset'].split(',')]
                image_urls.update(srcset_urls)  # Add all srcset URLs to the set

    # Output the collected image URLs
    if image_urls:
        print("Found the following unique image URLs:")
        for img_url in image_urls:
            print(img_url)
    else:
        print("No images found in the specified <div> elements.")
else:
    print(f"Failed to retrieve page: {response.status_code}")


Found the following unique image URLs:
https://i.ebayimg.com/images/g/KIoAAOSwrlBm8c55/s-l1600.jpg
https://i.ebayimg.com/images/g/xTkAAOSwrsFm8c5r/s-l1600.jpg
https://i.ebayimg.com/images/g/KO0AAOSwaX5m8c5x/s-l1600.jpg
https://i.ebayimg.com/images/g/rkAAAOSwMv9m8c53/s-l1600.jpg
https://i.ebayimg.com/images/g/54UAAOSwIvBm8c51/s-l1600.jpg
https://i.ebayimg.com/images/g/BJMAAOSwEP1m8c5w/s-l1600.jpg
https://i.ebayimg.com/images/g/zXYAAOSwUzZm8c5z/s-l1600.jpg
https://i.ebayimg.com/images/g/is4AAOSwhG5m8c55/s-l1600.jpg
https://i.ebayimg.com/images/g/d58AAOSw0PRm8c57/s-l1600.jpg
https://i.ebayimg.com/images/g/DSkAAOSw0H1m8c5q/s-l1600.jpg
https://i.ebayimg.com/images/g/TRgAAOSwzWdm8c50/s-l1600.jpg
https://i.ebayimg.com/images/g/E4UAAOSwaOJm8c56/s-l1600.jpg
https://i.ebayimg.com/images/g/p1UAAOSwQvdm8c5u/s-l1600.jpg
https://i.ebayimg.com/images/g/NfMAAOSwMB9m8c55/s-l1600.jpg
https://i.ebayimg.com/images/g/aNcAAOSwFHtm8c51/s-l1600.jpg
https://i.ebayimg.com/images/g/54kAAOSwIvBm8c53/s-l1600.jpg
h

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the eBay listing page
url = 'https://www.ebay.com/itm/135265261254?_skw=sedan&hash=item1f7e7016c6:g:Zo8AAOSwcCFm8c5b&itmprp=enc%3AAQAJAAAA4HoV3kP08IDx%2BKZ9MfhVJKkhtNc4zMnXPU9b4S7dbB%2BvOoqb9cE9OB2%2FllVZWzbjzjp4RL1RVC%2B9fAh2czCDxUl9dQDf5j2xA%2FQLWIhpBtu0%2BAKC7V9Tx5CbM4%2FWWA9IIZfboiMNolv8EcKoP0Hnh03m4BGuSszdj6Vue2cPBtpfcYvSNPanVq0W%2FVQDXQYKyNmQ%2BrUIWRePYCjMRHwoAhT6oKQhxwawqffD9714ot0zCtcXQglVBLxk7CmHHF%2Bjmd51FJYH3Ko4Est--dLCm9Xpw3D%2F6704joE5CuZLb7od%7Ctkp%3ABk9SR-qzjL_KZA'

# Send a GET request to the page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all <div> elements with the specified class that contains the image carousel items
    image_divs = soup.find_all('div', class_="ux-image-carousel-item image-treatment image")

    # Use a set to store unique image URLs
    image_urls = set()

    # Extract the URLs from the <img> tags within these <div> elements
    for div in image_divs:
        img_tag = div.find('img')  # Find the img tag within the div

        # Extract src, data-zoom-src, and srcset if available
        if img_tag:
            if 'src' in img_tag.attrs:
                image_urls.add(img_tag['src'])  # Add src to the set (avoiding duplicates)
            if 'data-zoom-src' in img_tag.attrs:
                image_urls.add(img_tag['data-zoom-src'])  # Add data-zoom-src to the set
            if 'srcset' in img_tag.attrs:
                # srcset can contain multiple resolutions; split them and add each one
                srcset_urls = [url.split()[0] for url in img_tag['srcset'].split(',')]
                image_urls.update(srcset_urls)  # Add all srcset URLs to the set

    # Convert the set of image URLs into a pandas DataFrame
    df = pd.DataFrame(list(image_urls), columns=["Image URL"])

    # Save the DataFrame to a CSV file
    df.to_csv('image_urls.csv', index=False)

    print(f"Image URLs have been saved to 'image_urls.csv'.")
else:
    print(f"Failed to retrieve page: {response.status_code}")


Image URLs have been saved to 'image_urls.csv'.


In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the eBay listing page
url = 'https://www.ebay.com/itm/135265261254?_skw=sedan&hash=item1f7e7016c6:g:Zo8AAOSwcCFm8c5b&itmprp=enc%3AAQAJAAAA4HoV3kP08IDx%2BKZ9MfhVJKkhtNc4zMnXPU9b4S7dbB%2BvOoqb9cE9OB2%2FllVZWzbjzjp4RL1RVC%2B9fAh2czCDxUl9dQDf5j2xA%2FQLWIhpBtu0%2BAKC7V9Tx5CbM4%2FWWA9IIZfboiMNolv8EcKoP0Hnh03m4BGuSszdj6Vue2cPBtpfcYvSNPanVq0W%2FVQDXQYKyNmQ%2BrUIWRePYCjMRHwoAhT6oKQhxwawqffD9714ot0zCtcXQglVBLxk7CmHHF%2Bjmd51FJYH3Ko4Est--dLCm9Xpw3D%2F6704joE5CuZLb7od%7Ctkp%3ABk9SR-qzjL_KZA'

# Send a GET request to the page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the car model name from the title (assuming it's inside an <h1> tag)
    car_model_tag = soup.find('h1', class_="x-item-title__mainTitle")
    car_model_name = car_model_tag.get_text(strip=True) if car_model_tag else "Unknown Car Model"

    # Find all <div> elements with the specified class that contains the image carousel items
    image_divs = soup.find_all('div', class_="ux-image-carousel-item image-treatment image")

    # Use a set to store unique image URLs
    image_urls = set()

    # Extract the URLs from the <img> tags within these <div> elements
    for div in image_divs:
        # Check if 'transform-origin' is in the style attribute
        if div.get('style') and 'transform-origin' in div['style']:
            img_tag = div.find('img')  # Find the img tag within the div

            # Extract src, data-zoom-src, and srcset if available and check if they end with .webp
            if img_tag:
                if 'src' in img_tag.attrs and img_tag['src'].endswith('.webp'):
                    image_urls.add(img_tag['src'])  # Add src to the set if it ends with .webp
                if 'data-zoom-src' in img_tag.attrs and img_tag['data-zoom-src'].endswith('.webp'):
                    image_urls.add(img_tag['data-zoom-src'])  # Add data-zoom-src if it ends with .webp
                if 'srcset' in img_tag.attrs:
                    # srcset can contain multiple resolutions; split them and add only those ending with .webp
                    srcset_urls = [url.split()[0] for url in img_tag['srcset'].split(',') if url.split()[0].endswith('.webp')]
                    image_urls.update(srcset_urls)  # Add all .webp URLs from srcset to the set

    # Convert the set of image URLs into a pandas DataFrame
    df = pd.DataFrame(list(image_urls), columns=["Image URL"])

    # Add the car model name as a new column and reorder so that 'Car Model' is first
    df.insert(0, 'Car Model', car_model_name)

    # Save the DataFrame to a CSV file
    df.to_csv('image_urls_with_model.csv', index=False)

    print(f"Image URLs and car model have been saved to 'image_urls_with_model.csv'.")
else:
    print(f"Failed to retrieve page: {response.status_code}")


Image URLs and car model have been saved to 'image_urls_with_model.csv'.


## 2.2 Saving the images URLs for 1 car model - improvements

Adding the car model name to the output csv file so that we can map with the ebay_sedan_info.csv.

In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the eBay listing page
url = 'https://www.ebay.com/itm/167071932782?_skw=sedan&hash=item26e643616e:g:-jwAAOSwhlhnLQBI&itmprp=enc%3AAQAJAAAAwHoV3kP08IDx%2BKZ9MfhVJKnWoWbAulwmirxgpAmc9vEbKzOB2BXTR7zZThRychIjMILuyreM2sSeDsgqHPiMFaWEVYViaFfB6rIPDYHH5fVXLi%2FSTZt7Qe7vgWIqVSWmyPTcUJwmu5ezQS5hHvWuJA2JMeQ98%2BIqex7xZFcYhCLFXOe9%2FHaUh2p0mEIsNgrRGtRZftuNRKshuuL2L6nrn6gTVASt01Z%2Fmjo%2FrqeaDjy1l%2Be5FLxdkgIzB1BMcdA8lQ%3D%3D%7Ctkp%3ABk9SR-S6kZfhZA'

# Send a GET request to the page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the car model name from the title (assuming it's inside an <h1> tag)
    car_model_tag = soup.find('h1', class_="x-item-title__mainTitle")
    car_model_name = car_model_tag.get_text(strip=True) if car_model_tag else "Unknown Car Model"

    # Find all <div> elements with the specified class that contains the image carousel items
    image_divs = soup.find_all('div', class_="ux-image-carousel-item image-treatment image")

    # Use a set to store unique image URLs
    image_urls = set()

    # Extract the URLs from the <img> tags within these <div> elements
    for div in image_divs:
        img_tag = div.find('img')  # Find the img tag within the div

        # Extract src, data-zoom-src, and srcset if available
        if img_tag:
            if 'src' in img_tag.attrs:
                image_urls.add(img_tag['src'])  # Add src to the set (avoiding duplicates)
            if 'data-zoom-src' in img_tag.attrs:
                image_urls.add(img_tag['data-zoom-src'])  # Add data-zoom-src to the set
            if 'srcset' in img_tag.attrs:
                # srcset can contain multiple resolutions; split them and add each one
                srcset_urls = [url.split()[0] for url in img_tag['srcset'].split(',')]
                image_urls.update(srcset_urls)  # Add all srcset URLs to the set

    # Convert the set of image URLs into a pandas DataFrame
    df = pd.DataFrame(list(image_urls), columns=["Image URL"])

    # Add the car model name as a new column and reorder so that 'Car Model' is first
    df.insert(0, 'Car Model', car_model_name)

    # Save the DataFrame to a CSV file
    df.to_csv('image_urls_with_model.csv', index=False)

    print(f"Image URLs and car model have been saved to 'image_urls_with_model.csv'.")
else:
    print(f"Failed to retrieve page: {response.status_code}")


Image URLs and car model have been saved to 'image_urls_with_model.csv'.


In [38]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the eBay listing page
url = 'https://www.ebay.com/itm/167071932782?_skw=sedan&hash=item26e643616e:g:-jwAAOSwhlhnLQBI&itmprp=enc%3AAQAJAAAAwHoV3kP08IDx%2BKZ9MfhVJKnWoWbAulwmirxgpAmc9vEbKzOB2BXTR7zZThRychIjMILuyreM2sSeDsgqHPiMFaWEVYViaFfB6rIPDYHH5fVXLi%2FSTZt7Qe7vgWIqVSWmyPTcUJwmu5ezQS5hHvWuJA2JMeQ98%2BIqex7xZFcYhCLFXOe9%2FHaUh2p0mEIsNgrRGtRZftuNRKshuuL2L6nrn6gTVASt01Z%2Fmjo%2FrqeaDjy1l%2Be5FLxdkgIzB1BMcdA8lQ%3D%3D%7Ctkp%3ABk9SR-S6kZfhZA'

# Send a GET request to the page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the car model name from the title (assuming it's inside an <h1> tag)
    car_model_tag = soup.find('h1', class_="x-item-title__mainTitle")
    car_model_name = car_model_tag.get_text(strip=True) if car_model_tag else "Unknown Car Model"

    # Find all <div> elements with the specified class that contains the image carousel items
    image_divs = soup.find_all('div', class_="ux-image-carousel-item image-treatment image")

    # Use a set to store unique image URLs
    image_urls = set()

    # Extract the URLs from the <img> tags within these <div> elements
    for div in image_divs:
        # Check if 'transform-origin' is in the style attribute
        #print(div)
        if div.get('style') and 'transform-origin' in div['style']:
            img_tag = div.find('img')  # Find the img tag within the div

            # Extract src, data-zoom-src, and srcset if available
            if img_tag:
                if 'src' in img_tag.attrs:
                    image_urls.add(img_tag['src'])  # Add src to the set (avoiding duplicates)
                if 'data-zoom-src' in img_tag.attrs:
                    image_urls.add(img_tag['data-zoom-src'])  # Add data-zoom-src to the set
                if 'srcset' in img_tag.attrs:
                    # srcset can contain multiple resolutions; split them and add each one
                    srcset_urls = [url.split()[0] for url in img_tag['srcset'].split(',')]
                    image_urls.update(srcset_urls)  # Add all srcset URLs to the set

    # Convert the set of image URLs into a pandas DataFrame
    df = pd.DataFrame(list(image_urls), columns=["Image URL"])

    # Add the car model name as a new column and reorder so that 'Car Model' is first
    df.insert(0, 'Car Model', car_model_name)

    # Save the DataFrame to a CSV file
    df.to_csv('image_urls_with_model.csv', index=False)

    print(f"Image URLs and car model have been saved to 'image_urls_with_model.csv'.")
else:
    print(f"Failed to retrieve page: {response.status_code}")


Image URLs and car model have been saved to 'image_urls_with_model.csv'.


In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Initialize Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# URL of the eBay listing page
url = 'https://www.ebay.com/itm/135265261254?_skw=sedan&hash=item1f7e7016c6:g:Zo8AAOSwcCFm8c5b&itmprp=enc%3AAQAJAAAA4HoV3kP08IDx%2BKZ9MfhVJKkhtNc4zMnXPU9b4S7dbB%2BvOoqb9cE9OB2%2FllVZWzbjzjp4RL1RVC%2B9fAh2czCDxUl9dQDf5j2xA%2FQLWIhpBtu0%2BAKC7V9Tx5CbM4%2FWWA9IIZfboiMNolv8EcKoP0Hnh03m4BGuSszdj6Vue2cPBtpfcYvSNPanVq0W%2FVQDXQYKyNmQ%2BrUIWRePYCjMRHwoAhT6oKQhxwawqffD9714ot0zCtcXQglVBLxk7CmHHF%2Bjmd51FJYH3Ko4Est--dLCm9Xpw3D%2F6704joE5CuZLb7od%7Ctkp%3ABk9SR-qzjL_KZA'

# Open the page
driver.get(url)

# Wait for the page to load completely by waiting for any image carousel items to become visible
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "ux-image-carousel-item")))
time.sleep(3)  # Additional wait for any dynamic JavaScript content

# Get the car model name (assuming it's inside an <h1> tag with class `x-item-title__mainTitle`)
try:
    car_model_tag = driver.find_element(By.CLASS_NAME, "x-item-title__mainTitle")
    car_model_name = car_model_tag.text.strip()
except:
    car_model_name = "Unknown Car Model"

# Use a set to store unique image URLs
image_urls = set()

# Find all image elements in the carousel
image_divs = driver.find_elements(By.CLASS_NAME, "ux-image-carousel-item")

for div in image_divs:
    try:
        # Attempt to find the img tag within the div; if not present, skip this div
        img_tag = div.find_element(By.TAG_NAME, 'img')
        
        # Check if 'transform-origin' is in the inline style attribute of the img tag
        inline_style = img_tag.get_attribute("style")
        if inline_style and 'transform-origin' in inline_style:
            # Collect image URLs from src, data-zoom-src, and srcset
            for attribute in ['src', 'data-zoom-src', 'srcset']:
                url = img_tag.get_attribute(attribute)
                if url:
                    # Handle srcset separately as it may contain multiple URLs
                    if attribute == 'srcset':
                        urls = url.split(',')
                        for srcset_url in urls:
                            srcset_url_clean = srcset_url.split()[0]  # Clean up any additional info
                            image_urls.add(srcset_url_clean)  # Add to the set
                    else:
                        image_urls.add(url)  # Add to the set
    except Exception as e:
        # Print debugging information
        print(f"Error processing image in div: {e}")
        print("HTML of the problematic div:")
        print(div.get_attribute('outerHTML'))

# Close the browser
driver.quit()

# Convert the set of image URLs into a pandas DataFrame
df = pd.DataFrame(list(image_urls), columns=["Image URL"])

# Add the car model name as a new column and reorder so that 'Car Model' is first
df.insert(0, 'Car Model', car_model_name)

# Save the DataFrame to a CSV file
df.to_csv('image_urls_with_model.csv', index=False)

print(f"Image URLs and car model have been saved to 'image_urls_with_model.csv'.")


Error processing image in div: Message: no such element: Unable to locate element: {"method":"tag name","selector":"img"}
  (Session info: chrome=130.0.6723.117); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001048c15dc cxxbridge1$str$ptr + 3653648
1   chromedriver                        0x00000001048b9e3c cxxbridge1$str$ptr + 3623024
2   chromedriver                        0x0000000104324100 cxxbridge1$string$len + 88404
3   chromedriver                        0x000000010436641c cxxbridge1$string$len + 359536
4   chromedriver                        0x000000010435ca58 cxxbridge1$string$len + 320172
5   chromedriver                        0x000000010439fba8 cxxbridge1$string$len + 594940
6   chromedriver                        0x000000010435b0fc cxxbridge1$string$len + 313680
7   chromedriver                        0x000000010435b

## 2.3 Saving the images URLs for all car models

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Load the CSV file with car model URLs
df = pd.read_csv('ebay_sedans_info.csv')

# Create an empty list to store the final data
car_images_data = []

# Loop through each row in the CSV file
for index, row in df.iterrows():
    car_model_name = row['Car Model']  # Get the car model name
    car_model_url = row['Car URL']     # Get the car model URL
    
    print(f"Scraping images for {car_model_name}...")

    # Send a GET request to the car model page
    response = requests.get(car_model_url)

    if response.status_code == 200:
        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all <div> elements with the specified class that contains the image carousel items
        image_divs = soup.find_all('div', class_="ux-image-carousel-item image-treatment image")

        # Use a set to store unique image URLs
        image_urls = set()

        # Extract the URLs from the <img> tags within these <div> elements
        for div in image_divs:
            img_tag = div.find('img')  # Find the img tag within the div

            # Extract src, data-zoom-src, and srcset if available
            if img_tag:
                if 'src' in img_tag.attrs:
                    image_urls.add(img_tag['src'])  # Add src to the set (avoiding duplicates)
                if 'data-zoom-src' in img_tag.attrs:
                    image_urls.add(img_tag['data-zoom-src'])  # Add data-zoom-src to the set
                if 'srcset' in img_tag.attrs:
                    # srcset can contain multiple resolutions; split them and add each one
                    srcset_urls = [url.split()[0] for url in img_tag['srcset'].split(',')]
                    image_urls.update(srcset_urls)  # Add all srcset URLs to the set

        # Append each image URL to the list with the car model name
        for img_url in image_urls:
            car_images_data.append({'Car Model': car_model_name, 'Image URL': img_url})

    else:
        print(f"Failed to retrieve page for {car_model_name}: {response.status_code}")

# Convert the list of dictionaries to a pandas DataFrame
car_images_df = pd.DataFrame(car_images_data)

# Save the DataFrame to a new CSV file
car_images_df.to_csv('all_car_images.csv', index=False)

print("All image URLs have been saved to 'all_car_images.csv'.")


Scraping images for Buick: Grand National GRAND NATIONAL...
Scraping images for 1956 CITROEN ...
Scraping images for 2016 BMW 7-Series ...
Scraping images for 1995 Chevrolet Impala ...
Scraping images for Audi: A4 S-Line...
Scraping images for 2007 Mercedes-Benz S-Class S65...
Scraping images for Mercedes-Benz: E-Class...
Scraping images for BMW: 5-Series...
Scraping images for 1996 Chevrolet Impala ...
Scraping images for 1987 Honda Civic 1.5...
Scraping images for 1986 Mercedes-Benz 190-Series ...
Scraping images for Infiniti: G37...
Scraping images for Mercedes-Benz: S-Class Lorinser...
Scraping images for Cadillac: Fleetwood...
Scraping images for Mitsubishi: Lancer...
Scraping images for BMW: 3-Series 320i Xdrive...
Scraping images for Dodge: Sedan...
Scraping images for Chevrolet: Bel Air/150/210 210...
Scraping images for Tesla: Model S...
Scraping images for Toyota: Supra...
Scraping images for Cadillac: Lasalle Stunning Black...
Scraping images for Toyota: Corolla CE...
Scrapi

Scraping images for 2014 Volkswagen Jetta ...
Scraping images for 2016 Bentley Mulsanne SPEED...
Scraping images for 1969 Ford Country Sedan Country sedan...
Scraping images for 2020 Cadillac CT4 -V Sedan...
Scraping images for 2017 Hyundai Elantra Limited...
Scraping images for 2021 Mercedes-Benz S-Class S 580 4MATIC Sedan 4D...
Scraping images for 1964 Chevrolet Impala ...
Scraping images for 2024 Hyundai Genesis AWD 3.5T SPORT ADVANCED-EDITION(ALL OPTIONS)...
Scraping images for 1970 Plymouth Fury ...
Scraping images for 2006 Chrysler 300 Series ...
Scraping images for 1967 Chevrolet Impala 4 door hardtop...
Scraping images for 2013 Tesla Model S ...
Scraping images for 2007 Toyota Corolla CE...
Scraping images for 2017 Lincoln Continental Black Label AWD 4dr Sedan...
Scraping images for 2016 Ford Taurus ...
Scraping images for 2019 Lexus LS LS500...
Scraping images for 2019 BMW M5 Base AWD 4dr Sedan...
Scraping images for 1939 Ford Deluxe base...
Scraping images for 2013 Audi S7 4.

Scraping images for 2006 Bentley Arnage 4dr Sedan T...
Scraping images for 2019 BMW 7-Series M760i xDrive AWD 4dr Sedan...
Scraping images for 2008 BMW M5 ...
Scraping images for 2024 Lexus ES 350 F Sport...
Scraping images for 2013 Honda Accord LX...
Scraping images for 2003 Buick LeSabre LIMITED...
Scraping images for 2023 Mercedes-Benz E-Class AMG E 63 S 4MATIC+ Sedan...
Scraping images for 2020 Cadillac CT6 LUXURY...
Scraping images for 2023 LUCID Air Pure $88525 MSRP FREE CHARGING UNTIL JUNE 2026! ...
Scraping images for 2005 Lincoln Town Car Signature Limited 4dr Sedan...
Scraping images for 2015 Audi A8 3.0T quattro AWD 4dr Sedan...
Scraping images for 2018 Honda Accord ...
Scraping images for 2014 BMW 5-Series XI...
Scraping images for 2016 Tesla Model S ...
Scraping images for 2008 Maserati Quattroporte M139...
Scraping images for 1968 Chevrolet Chevelle Malibu...
Scraping images for 1967 Chevrolet Impala 4 door hardtop...
Scraping images for 2021 BMW M3 CARBON BUCKETS / EXEC 

Scraping images for 2015 Volkswagen Passat SEL...
Scraping images for 1999 Mercedes-Benz E-Class E 320...
Scraping images for 1940 Ford Sedan ...
Scraping images for 1959 Ford Galaxie ...
Scraping images for 1993 Cadillac Fleetwood 5.7L V8 Brougham...
Scraping images for 2007 Cadillac DTS Luxury II 4dr Sedan...
Scraping images for 2018 Porsche Panamera Turbo...
Scraping images for 2016 Subaru WRX Premium...
Scraping images for 2009 Jaguar XJ VANDEN PLAS - 20K MILES - ONE OWNER - PRISTINE!...
Scraping images for 2002 Lexus IS IS300 5MT - 2-Owner - SoCal...
Scraping images for 2017 Ford Fusion SE...
Scraping images for 1989 Jaguar XJ6 ...
Scraping images for 1991 BMW M5 ...
Scraping images for 1986 Chevrolet Caprice CLASSIC...
Scraping images for 2012 INFINITI M M37 Sedan 4D...
Scraping images for 2017 BMW 3-Series ...
Scraping images for 2008 INFINITI M35 ...
Scraping images for 2002 Jaguar XJ XJ8 4dr Sedan...
Scraping images for 1954 Oldsmobile Eighty-Eight ...
Scraping images for 2022

Scraping images for 2016 Buick Lacrosse PREMIUM I ULTRA LUXURY, COMFORT & DRIVER CONFIDENCE PACKAGE...
Scraping images for 2019 Honda Insight TOURING...
Scraping images for 2025 Chevrolet Malibu LS...
Scraping images for 2010 BMW 5-Series ...
Scraping images for 1931 Ford Model A ...
Scraping images for 1966 Pontiac Catalina ...
Scraping images for 2018 Subaru Impreza SPORT...
Scraping images for 2003 BMW 3-Series XI...
Scraping images for 2022 Cadillac BLACKWING 6.2L SUPERCHARGED V8 DI Navigation/Leather CARBON FIBER...
Scraping images for 2024 Mercedes-Benz S-Class AMG® S 63 E Performance...
Scraping images for 2016 BMW 5-Series 535i 4dr Sedan...
Scraping images for 2019 Tesla Model 3 ...
Scraping images for 1988 BMW 3-Series I AUTOMATIC...
Scraping images for 2021 Porsche Panamera 4...
Scraping images for 2021 Porsche Taycan Turbo S...
Scraping images for 2017 BMW 7-Series ...
Scraping images for 1997 Lincoln Town Car SIGNATURE...
Scraping images for 2007 BMW 3-Series ...
Scraping i

# 3. Download the images using the URLs from last step

In [11]:
import os
import pandas as pd
import requests

# Load the CSV with the image URLs
df = pd.read_csv('all_car_images.csv')

# Create a directory to save the images if it doesn't exist
output_folder = 'downloaded_images'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loop through each image URL and download it
for index, row in df.iterrows():
    image_url = row['Image URL']

    # Try to download the image
    try:
        response = requests.get(image_url, stream=True)
        if response.status_code == 200:
            # Extract the file name from the URL
            filename = os.path.join(output_folder, f'image_{index+1}.jpg')
            # Write the image to the output folder
            with open(filename, 'wb') as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            print(f"Downloaded {filename}")
        else:
            print(f"Failed to download {image_url} - Status code: {response.status_code}")
    except Exception as e:
        print(f"Error downloading {image_url}: {e}")

print("All downloads completed.")


Downloaded downloaded_images/image_1.jpg
Downloaded downloaded_images/image_2.jpg
Downloaded downloaded_images/image_3.jpg
Downloaded downloaded_images/image_4.jpg
Downloaded downloaded_images/image_5.jpg
Downloaded downloaded_images/image_6.jpg
Downloaded downloaded_images/image_7.jpg
Downloaded downloaded_images/image_8.jpg
Downloaded downloaded_images/image_9.jpg
Downloaded downloaded_images/image_10.jpg
Downloaded downloaded_images/image_11.jpg
Downloaded downloaded_images/image_12.jpg
Downloaded downloaded_images/image_13.jpg
Downloaded downloaded_images/image_14.jpg
Downloaded downloaded_images/image_15.jpg
Downloaded downloaded_images/image_16.jpg
Downloaded downloaded_images/image_17.jpg
Downloaded downloaded_images/image_18.jpg
Downloaded downloaded_images/image_19.jpg
Downloaded downloaded_images/image_20.jpg
Downloaded downloaded_images/image_21.jpg
Downloaded downloaded_images/image_22.jpg
Downloaded downloaded_images/image_23.jpg
Downloaded downloaded_images/image_24.jpg
D

Downloaded downloaded_images/image_195.jpg
Downloaded downloaded_images/image_196.jpg
Downloaded downloaded_images/image_197.jpg
Downloaded downloaded_images/image_198.jpg
Downloaded downloaded_images/image_199.jpg
Downloaded downloaded_images/image_200.jpg
Downloaded downloaded_images/image_201.jpg
Downloaded downloaded_images/image_202.jpg
Downloaded downloaded_images/image_203.jpg
Downloaded downloaded_images/image_204.jpg
Downloaded downloaded_images/image_205.jpg
Downloaded downloaded_images/image_206.jpg
Downloaded downloaded_images/image_207.jpg
Downloaded downloaded_images/image_208.jpg
Downloaded downloaded_images/image_209.jpg
Downloaded downloaded_images/image_210.jpg
Downloaded downloaded_images/image_211.jpg
Downloaded downloaded_images/image_212.jpg
Downloaded downloaded_images/image_213.jpg
Downloaded downloaded_images/image_214.jpg
Downloaded downloaded_images/image_215.jpg
Downloaded downloaded_images/image_216.jpg
Downloaded downloaded_images/image_217.jpg
Downloaded 

Downloaded downloaded_images/image_386.jpg
Downloaded downloaded_images/image_387.jpg
Downloaded downloaded_images/image_388.jpg
Downloaded downloaded_images/image_389.jpg
Downloaded downloaded_images/image_390.jpg
Downloaded downloaded_images/image_391.jpg
Downloaded downloaded_images/image_392.jpg
Downloaded downloaded_images/image_393.jpg
Downloaded downloaded_images/image_394.jpg
Downloaded downloaded_images/image_395.jpg
Downloaded downloaded_images/image_396.jpg
Downloaded downloaded_images/image_397.jpg
Downloaded downloaded_images/image_398.jpg
Downloaded downloaded_images/image_399.jpg
Downloaded downloaded_images/image_400.jpg
Downloaded downloaded_images/image_401.jpg
Downloaded downloaded_images/image_402.jpg
Downloaded downloaded_images/image_403.jpg
Downloaded downloaded_images/image_404.jpg
Downloaded downloaded_images/image_405.jpg
Downloaded downloaded_images/image_406.jpg
Downloaded downloaded_images/image_407.jpg
Downloaded downloaded_images/image_408.jpg
Downloaded 

Downloaded downloaded_images/image_577.jpg
Downloaded downloaded_images/image_578.jpg
Downloaded downloaded_images/image_579.jpg
Downloaded downloaded_images/image_580.jpg
Downloaded downloaded_images/image_581.jpg
Downloaded downloaded_images/image_582.jpg
Downloaded downloaded_images/image_583.jpg
Downloaded downloaded_images/image_584.jpg
Downloaded downloaded_images/image_585.jpg
Downloaded downloaded_images/image_586.jpg
Downloaded downloaded_images/image_587.jpg
Downloaded downloaded_images/image_588.jpg
Downloaded downloaded_images/image_589.jpg
Downloaded downloaded_images/image_590.jpg
Downloaded downloaded_images/image_591.jpg
Downloaded downloaded_images/image_592.jpg
Downloaded downloaded_images/image_593.jpg
Downloaded downloaded_images/image_594.jpg
Downloaded downloaded_images/image_595.jpg
Downloaded downloaded_images/image_596.jpg
Downloaded downloaded_images/image_597.jpg
Downloaded downloaded_images/image_598.jpg
Downloaded downloaded_images/image_599.jpg
Downloaded 

Downloaded downloaded_images/image_769.jpg
Downloaded downloaded_images/image_770.jpg
Downloaded downloaded_images/image_771.jpg
Downloaded downloaded_images/image_772.jpg
Downloaded downloaded_images/image_773.jpg
Downloaded downloaded_images/image_774.jpg
Downloaded downloaded_images/image_775.jpg
Downloaded downloaded_images/image_776.jpg
Downloaded downloaded_images/image_777.jpg
Downloaded downloaded_images/image_778.jpg
Downloaded downloaded_images/image_779.jpg
Downloaded downloaded_images/image_780.jpg
Downloaded downloaded_images/image_781.jpg
Downloaded downloaded_images/image_782.jpg
Downloaded downloaded_images/image_783.jpg
Downloaded downloaded_images/image_784.jpg
Downloaded downloaded_images/image_785.jpg
Downloaded downloaded_images/image_786.jpg
Downloaded downloaded_images/image_787.jpg
Downloaded downloaded_images/image_788.jpg
Downloaded downloaded_images/image_789.jpg
Downloaded downloaded_images/image_790.jpg
Downloaded downloaded_images/image_791.jpg
Downloaded 

...

Downloaded downloaded_images/image_9562.jpg
Downloaded downloaded_images/image_9563.jpg
Downloaded downloaded_images/image_9564.jpg
Downloaded downloaded_images/image_9565.jpg
Downloaded downloaded_images/image_9566.jpg
Downloaded downloaded_images/image_9567.jpg
Downloaded downloaded_images/image_9568.jpg
Downloaded downloaded_images/image_9569.jpg
Downloaded downloaded_images/image_9570.jpg
Downloaded downloaded_images/image_9571.jpg
Downloaded downloaded_images/image_9572.jpg
Downloaded downloaded_images/image_9573.jpg
Downloaded downloaded_images/image_9574.jpg
Downloaded downloaded_images/image_9575.jpg
Downloaded downloaded_images/image_9576.jpg
Downloaded downloaded_images/image_9577.jpg
Downloaded downloaded_images/image_9578.jpg
Downloaded downloaded_images/image_9579.jpg
Downloaded downloaded_images/image_9580.jpg
Downloaded downloaded_images/image_9581.jpg
Downloaded downloaded_images/image_9582.jpg
Downloaded downloaded_images/image_9583.jpg
Downloaded downloaded_images/ima

Downloaded downloaded_images/image_9750.jpg
Downloaded downloaded_images/image_9751.jpg
Downloaded downloaded_images/image_9752.jpg
Downloaded downloaded_images/image_9753.jpg
Downloaded downloaded_images/image_9754.jpg
Downloaded downloaded_images/image_9755.jpg
Downloaded downloaded_images/image_9756.jpg
Downloaded downloaded_images/image_9757.jpg
Downloaded downloaded_images/image_9758.jpg
Downloaded downloaded_images/image_9759.jpg
Downloaded downloaded_images/image_9760.jpg
Downloaded downloaded_images/image_9761.jpg
Downloaded downloaded_images/image_9762.jpg
Downloaded downloaded_images/image_9763.jpg
Downloaded downloaded_images/image_9764.jpg
Downloaded downloaded_images/image_9765.jpg
Downloaded downloaded_images/image_9766.jpg
Downloaded downloaded_images/image_9767.jpg
Downloaded downloaded_images/image_9768.jpg
Downloaded downloaded_images/image_9769.jpg
Downloaded downloaded_images/image_9770.jpg
Downloaded downloaded_images/image_9771.jpg
Downloaded downloaded_images/ima

Downloaded downloaded_images/image_9937.jpg
Downloaded downloaded_images/image_9938.jpg
Downloaded downloaded_images/image_9939.jpg
Downloaded downloaded_images/image_9940.jpg
Downloaded downloaded_images/image_9941.jpg
Downloaded downloaded_images/image_9942.jpg
Downloaded downloaded_images/image_9943.jpg
Downloaded downloaded_images/image_9944.jpg
Downloaded downloaded_images/image_9945.jpg
Downloaded downloaded_images/image_9946.jpg
Downloaded downloaded_images/image_9947.jpg
Downloaded downloaded_images/image_9948.jpg
Downloaded downloaded_images/image_9949.jpg
Downloaded downloaded_images/image_9950.jpg
Downloaded downloaded_images/image_9951.jpg
Downloaded downloaded_images/image_9952.jpg
Downloaded downloaded_images/image_9953.jpg
Downloaded downloaded_images/image_9954.jpg
Downloaded downloaded_images/image_9955.jpg
Downloaded downloaded_images/image_9956.jpg
Downloaded downloaded_images/image_9957.jpg
Downloaded downloaded_images/image_9958.jpg
Downloaded downloaded_images/ima

Downloaded downloaded_images/image_10122.jpg
Downloaded downloaded_images/image_10123.jpg
Downloaded downloaded_images/image_10124.jpg
Downloaded downloaded_images/image_10125.jpg
Downloaded downloaded_images/image_10126.jpg
Downloaded downloaded_images/image_10127.jpg
Downloaded downloaded_images/image_10128.jpg
Downloaded downloaded_images/image_10129.jpg
Downloaded downloaded_images/image_10130.jpg
Downloaded downloaded_images/image_10131.jpg
Downloaded downloaded_images/image_10132.jpg
Downloaded downloaded_images/image_10133.jpg
Downloaded downloaded_images/image_10134.jpg
Downloaded downloaded_images/image_10135.jpg
Downloaded downloaded_images/image_10136.jpg
Downloaded downloaded_images/image_10137.jpg
Downloaded downloaded_images/image_10138.jpg
Downloaded downloaded_images/image_10139.jpg
Downloaded downloaded_images/image_10140.jpg
Downloaded downloaded_images/image_10141.jpg
Error downloading nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Downloa

KeyboardInterrupt: 